<a href="https://colab.research.google.com/github/RIPS-2024-Aerospace/Aerospace-Project/blob/main/BFGStestStandardOptimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 352 (delta 74), reused 44 (delta 44), pack-reused 269
Receiving objects: 100% (352/352), 20.91 MiB | 13.00 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

np.random.seed(163)

%run "/content/Aerospace-Project/Standard Filters/DiffKf.ipynb"
%run "/content/Aerospace-Project/Standard Filters/KF.ipynb"

In [69]:
class Node:
    def __init__(self,id,F,G,H,R,Q,x0,P,q): #q = quality (as a percent, 1 = best)
        self.id = id
        self.F = F
        self.G = G
        self.H = H
        self.R = R
        self.Q = Q
        self.q = q

        #Has an instance for each node
        self.x = x0
        self.n = len(x0)

        self.P = P

        self.nbhrs = []
        self.nbhr_weights = {}

        self.psi = np.copy(x0)

    def predict(self):
        self.x = self.F@(np.sum([self.nbhr_weights[node.id]*node.psi for node in self.nbhrs],0))+(1-self.q)*50
        self.P = (self.F@self.P@self.F.T) + (self.G@self.Q@self.G.T)*(1/self.q)

    def update(self, y):
        S = lambda node: (node.H @self.P @ node.H.T) + node.R
        #Original Code for K
        #K = {node.id: self.P@ node.H.T @ np.linalg.inv(S(node)) for node in self.nbhrs}
        #We use a solver to avoid inverses:
        #K[i]@S = P@H[i].T --> S.T@K[i].T = H[i]@P.T --> cholesky solve for K[i]
        K = {node.id:sp.linalg.cho_solve(sp.linalg.cho_factor(S(node).T),(node.H @ self.P.T)).T for node in self.nbhrs}

        I = lambda node: y[node.id] - (node.H @self.psi)
        self.psi = self.x

        for node in self.nbhrs:
            self.psi = self.psi+(K[node.id]@I(node))

        for node in self.nbhrs:
            self.P = (np.eye(self.n,self.n) - K[node.id]@node.H)@self.P


class DiffKF:
    def __init__(self,C,F,G,H,R,Q,x0,P):

        #Number of nodes
        self.n = len(x0)

        #weighted adjacencey matrix, nodes must be connected to themselves
        self.C = C

        self.nodes = []
        #k = 0
        #print(self.n)
        for i in range(self.n):
            #print(k)
            self.nodes.append(Node(i,F[i],G[i],H[i],R[i],Q[i],x0[i],P[i], 1))
            #k+=1
        #print(k)
        #adding 2 "bad" nodes with quality
        #self.nodes.append(Node(k,F[k],G[k],H[k],R[k],Q[k],x0[k],P[k], 0.01))
        #k+=1
        #self.nodes.append(Node(k,F[k],G[k],H[k],R[k],Q[k],x0[k],P[k], 0.01))

        for i in range(self.n):
            for j in range(self.n):
                if self.C[i][j] != 0:
                    self.nodes[i].nbhrs.append(self.nodes[j])
                    self.nodes[i].nbhr_weights[j] = C[i][j]

    def predict(self): #a.k.a diffusion update
        result = []
        for node in self.nodes:
            node.predict()
            result.append(node.x)

        return result


    def update(self, y, nodes = [0,1,2,3,4]): #a.k.a Incremental update
        for i,node in enumerate(self.nodes):
            if i in nodes:
              node.update(y)
            else:
              node.update(np.zeros_like(y))

In [68]:
#Convert list of weights at each edge to a numpy adjacency matrix
def convert(weights):
  #C just is a reference for where we know there are edges in our network
  C = np.array([[0.34,0.33, 0, 0, 0.33],[0.33,0.34,0.33,0,0],[0,0.33,0.34,0.33,0],[0,0,0.33,0.34,0.33],[0.33,0,0,0.33,0.34]])

  W = []
  i = 0
  for arr in C:
    for val in arr:
      if val != 0:
        W.append(weights[i])
        i += 1
      else: W.append(0)
  C = np.array([[W[i] for i in range(0,5)], [W[i] for i in range(5,10)], [W[i] for i in range(10,15)], [W[i] for i in range(15,20)], [W[i] for i in range(20,25)]])
  return(C)

In [105]:
def run_filters(weights):
  dt = 10

  C = convert(weights)
  print(C)
  C_unweighted = np.array([[1 if x!=0 else 0 for x in row] for row in C])

  num_stns = len(C[0])

  A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
  H = np.array([[1, 0, 0, 0],[0,0,1,0]])

  dkf_state_size = len(A)
  dkf_measure_size = len(H)

  q = 0.001
  Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
  R = np.array([[4,0],[0,4]])

  A_kf = np.kron(np.eye(num_stns),A)
  H_kf = np.kron(np.eye(num_stns),H)
  Q_kf = np.kron(np.eye(num_stns),Q)
  R_kf = np.kron(np.eye(num_stns),R)

  kf_state_size = A_kf.shape[0]
  kf_measure_size = R_kf.shape[0]

  F = [A for _ in range(num_stns)]
  G = [np.eye(dkf_state_size) for _ in range(num_stns)]
  H_dkf = [H for _ in range(num_stns)]

  Q_dkf = [Q for _ in range(num_stns)]
  R_dkf = [R for _ in range(num_stns)]

  procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
  measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

  measure_kf_to_dkf  = lambda z: [np.array([z[dkf_measure_size*i + j] for j in range(dkf_measure_size)]) for i in range(num_stns)]
  state_kf_to_dkf = lambda z: [np.array([z[dkf_state_size*i + j] for j in range(dkf_state_size)]) for i in range(num_stns)]

  #True Initial
  x0_kf = np.array([[np.random.normal(0,np.sqrt(Q_kf[i,i])) for i in range(kf_state_size)]]).T


  #Initial Estimate
  x_kf = np.array([[np.random.normal(0,5) for i in range(kf_state_size)]]).T
  x_dkf = state_kf_to_dkf(x_kf)


  P_kf = 10*np.copy(Q_kf)
  P_dkf = [10*np.copy(Q) for _ in range(num_stns)]

  kf = KalmanFilter(A = A_kf,H = H_kf, Q = Q_kf, R = R_kf,P=P_kf,x0=x_kf)

  dkf = DiffKF(C,F,G,H_dkf,R_dkf,Q_dkf,x_dkf,P_dkf)

  iters = 60

  truth = np.zeros((iters+1,kf_state_size,1))
  truth[0] = x0_kf

  measurements = np.zeros((iters+1,kf_measure_size,1))
  measurements[0] = (H_kf @ x0_kf)+measure_noise_kf()


  predictions_kf = np.zeros((iters,kf_state_size,1))
  predictions_dkf = np.zeros((iters,num_stns,dkf_state_size,1))

  errors_kf = np.zeros((iters,kf_state_size,1))
  errors_dkf = np.zeros((iters,num_stns,dkf_state_size,1))

  P_hist_kf = np.zeros((iters,kf_state_size,kf_state_size))
  P_hist_dkf = np.zeros((iters, num_stns, dkf_state_size,dkf_state_size))


  full_system_P_hist = np.zeros((iters,kf_state_size,kf_state_size))
  prev_cov = np.block([[np.zeros(P_dkf[0].shape) if i!= j else dkf.nodes[i].P for j in range(num_stns)] for i in range(num_stns)])

  def get_diff_cov(prev_cov, Station_cov):

      S = lambda i: np.sum([node.H.T @ np.linalg.inv(node.R) @ node.H for node in dkf.nodes[i].nbhrs],axis = 0)

      S_full = np.block([[np.zeros(A.shape) if i!= j else S(j) for j in range(num_stns)] for i in range(num_stns)])
      H_full = np.kron(np.eye(num_stns),H)
      P_full = np.block([[np.zeros(P_dkf[0].shape) if i!= j else Station_cov[j] for j in range(num_stns)] for i in range(num_stns)])
      R_full = np.kron(np.eye(num_stns),R)


      C_full = np.kron(C,np.eye(dkf_state_size))
      A_full = np.kron(C_unweighted, np.eye(dkf_state_size))

      # Sigma1
      # compute the covariance (equation 32)
      F_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns),A)
      G_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns),G[0])
      D_i = C_full.T @ P_full @ A_full.T @ H_full.T @ np.linalg.inv(R_full)


      term1 = (F_i @ prev_cov @ F_i.T)
      term2 = G_i @ np.kron(np.ones((num_stns,num_stns)),Q)@G_i.T
      term3 = D_i @ R_full@D_i.T

      return term1 + term2 + term3

  for i in range(iters):

      kf.update(measurements[i])
      dkf.update(measure_kf_to_dkf(measurements[i]), [0,1,2,3,4])

      predictions_dkf[i] = [dkf.nodes[j].x for j in range(num_stns)]
      errors_dkf[i] = [dkf.nodes[j].x-state_kf_to_dkf(truth[i])[j] for j in range(num_stns)]
      station_covs = [dkf.nodes[j].P for j in range(num_stns)]
      P_hist_dkf[i] = station_covs

      prev_cov = get_diff_cov(prev_cov, station_covs)
      full_system_P_hist[i] = prev_cov

      predictions_kf[i] = kf.x
      errors_kf[i] = kf.x-truth[i]
      P_hist_kf[i] = kf.P

      kf.predict()
      dkf.predict()

      truth[i+1] = A_kf@x0_kf + procc_noise_kf()
      measurements[i+1] = H_kf @ truth[i+1] + measure_noise_kf()

  mu1 = np.zeros(kf_state_size,int)
  mu2 = np.zeros(kf_state_size, int)
  bhat = bhattacharyya_distance(mu1, mu2, full_system_P_hist[40], P_hist_kf[40])
  print(bhat)
  return(bhat)


In [51]:
def bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2):
    # mu1 = mean of diffusion KF
    # mu2 = mean of centralized KF
    # Sigma1 = covariance of diffusion KF
    # Sigma2 = covariance of centralized KF
    Sigma = (Sigma1 + Sigma2) / 2
    inv_Sigma = np.linalg.inv(Sigma)

    term1 = 1/8 * np.dot(np.dot((mu1 - mu2).T, inv_Sigma), (mu1 - mu2))
    term2 = 1/2 * np.log(np.linalg.det(Sigma) / np.sqrt(np.linalg.det(Sigma1) * np.linalg.det(Sigma2)))

    return term1 + term2

In [103]:
import scipy as sp
from scipy.optimize import minimize

def cost_func(diffusion_weights):
  return run_filters(diffusion_weights)


def run_optimize():
  C = np.array([[0.34,0.33, 0, 0, 0.33],[0.33,0.34,0.33,0,0],[0,0.33,0.34,0.33,0],[0,0,0.33,0.34,0.33],[0.33,0,0,0.33,0.34]])
  weights = []
  for arr in C:
    for val in arr:
      if val != 0:
        weights.append(val)

  run_filters(weights)
  x0 = np.array(weights)
  bounds = sp.optimize.Bounds(lb = np.zeros(len(weights)), ub = np.ones(len(weights)))
  result = sp.optimize.minimize(cost_func, x0, method='BFGS', bounds = bounds)
  print(result.x)


In [106]:
run_optimize()

[[0.34 0.33 0.   0.   0.33]
 [0.33 0.34 0.33 0.   0.  ]
 [0.   0.33 0.34 0.33 0.  ]
 [0.   0.   0.33 0.34 0.33]
 [0.33 0.   0.   0.33 0.34]]
22.66026062408553
[[0.34 0.33 0.   0.   0.33]
 [0.33 0.34 0.33 0.   0.  ]
 [0.   0.33 0.34 0.33 0.  ]
 [0.   0.   0.33 0.34 0.33]
 [0.33 0.   0.   0.33 0.34]]


<ipython-input-103-dfa2c7cfe91e>:19: RuntimeWarning: Method BFGS cannot handle bounds.
  result = sp.optimize.minimize(cost_func, x0, method='BFGS', bounds = bounds)


22.66026062408553
[[0.34000001 0.33       0.         0.         0.33      ]
 [0.33       0.34       0.33       0.         0.        ]
 [0.         0.33       0.34       0.33       0.        ]
 [0.         0.         0.33       0.34       0.33      ]
 [0.33       0.         0.         0.33       0.34      ]]
22.660260690126677
[[0.34       0.33000001 0.         0.         0.33      ]
 [0.33       0.34       0.33       0.         0.        ]
 [0.         0.33       0.34       0.33       0.        ]
 [0.         0.         0.33       0.34       0.33      ]
 [0.33       0.         0.         0.33       0.34      ]]
22.66026054572308
[[0.34       0.33       0.         0.         0.33000001]
 [0.33       0.34       0.33       0.         0.        ]
 [0.         0.33       0.34       0.33       0.        ]
 [0.         0.         0.33       0.34       0.33      ]
 [0.33       0.         0.         0.33       0.34      ]]
22.660260545740663
[[0.34       0.33       0.         0.         0.33   

KeyboardInterrupt: 